In [101]:
from bs4 import BeautifulSoup
import wikipedia
import requests
import pandas as pd
import csv
import time
import tqdm
import json
import sqlite3
import sqlalchemy as db
import functions as f

In [30]:
engine = db.create_engine('sqlite:///stocks.db')

In [31]:
frame_list = []
fails = []
sucess = []

In [32]:
sp500 = pd.read_csv('sp500.csv')

In [33]:
symbols = sp500.Symbol

In [39]:
for symbol in tqdm.tqdm(symbols[201:]):
    symbol = symbol.strip()
    req = requests.get(f'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY_ADJUSTED&symbol={symbol}&outputsize=full&apikey={key}')
    
    try:
        if req.status_code == 200:
            data = json.loads(req.text)
            df = df = pd.DataFrame(data['Time Series (Daily)']).transpose()
            df['symbol'] = symbol
            df.columns = map(str.lower, df.columns)
            df.to_sql(symbol, con=engine)
            frame_list.append(df.copy())
            sucess.append(symbol)
            time.sleep(20)
        else:
            fails.append(symbol)
        
    except:
        fails.append(symbol)

100%|██████████| 304/304 [2:03:06<00:00,  8.23s/it]  


In [207]:
# cleaning
formatted_frames = []
for df in frame_list:
    df = f.reformat_columns(df)
    df = f.typecast_df(df)
    df = f.rearrange_cols(df)
    df = f.date_to_column(df)
    x = df.copy()
    close_df =  x[['date', 'close']][0:1390] #create df with just close price
    close_df.columns = [['date', x.symbol[0]]]
    formatted_frames.append(close_df)
    

In [216]:
type(list(joined_df.columns)[0])

tuple

In [208]:
for df in formatted_frames[1:]:
     joined_df = joined_df.merge(df, how='left')
joined_df.dropna(axis='columns', inplace=True)
    

In [217]:
joined_df.dropna(axis='columns', inplace=True)
joined_df.head()

,date,MMM,AOS,ABT,ABBV,ACN,ATVI,AYI,ADBE,AAP,...,WU,WY,WHR,WMB,WYNN,XEL,XRX,XLNX,XYL,YUM
0,2019-07-12,173.34,45.19,83.92,70.28,196.21,47.20,130.64,309.42,158.58,...,20.50,25.54,146.42,28.85,133.89,60.80,35.37,120.32,82.50,111.05
1,2019-07-11,169.04,44.32,85.77,71.24,194.67,47.34,128.01,307.60,156.29,...,20.41,25.81,142.63,28.71,130.65,61.19,34.92,118.37,81.15,111.50
2,2019-07-10,165.70,44.77,85.38,71.65,191.71,46.84,127.95,306.98,156.43,...,20.55,26.31,142.08,28.82,130.34,61.17,35.48,117.49,80.39,110.98
3,2019-07-09,165.70,45.06,85.27,71.24,191.09,46.37,129.98,304.20,155.50,...,20.65,26.29,142.80,28.74,133.73,60.89,35.95,117.34,80.98,110.49
4,2019-07-08,169.19,45.21,85.09,71.22,190.31,46.47,131.47,302.92,158.19,...,20.35,26.43,144.53,28.44,133.06,60.85,35.66,115.93,81.70,110.05


In [221]:
new_col_names = []
for x in joined_df.columns:
    new_col_names.append(x[0])
joined_df.columns = new_col_names

In [225]:
engine = db.create_engine('sqlite:///stocks_formatted.db')

In [226]:
joined_df.to_sql('price_close', con=engine)

In [227]:
engine.execute("PRAGMA table_info(price_close)").fetchall()

[(0, 'index', 'BIGINT', 0, None, 0),
 (1, 'date', 'TEXT', 0, None, 0),
 (2, 'MMM', 'FLOAT', 0, None, 0),
 (3, 'AOS', 'FLOAT', 0, None, 0),
 (4, 'ABT', 'FLOAT', 0, None, 0),
 (5, 'ABBV', 'FLOAT', 0, None, 0),
 (6, 'ACN', 'FLOAT', 0, None, 0),
 (7, 'ATVI', 'FLOAT', 0, None, 0),
 (8, 'AYI', 'FLOAT', 0, None, 0),
 (9, 'ADBE', 'FLOAT', 0, None, 0),
 (10, 'AAP', 'FLOAT', 0, None, 0),
 (11, 'AMD', 'FLOAT', 0, None, 0),
 (12, 'AES', 'FLOAT', 0, None, 0),
 (13, 'AMG', 'FLOAT', 0, None, 0),
 (14, 'AFL', 'FLOAT', 0, None, 0),
 (15, 'A', 'FLOAT', 0, None, 0),
 (16, 'APD', 'FLOAT', 0, None, 0),
 (17, 'AKAM', 'FLOAT', 0, None, 0),
 (18, 'ALK', 'FLOAT', 0, None, 0),
 (19, 'ALB', 'FLOAT', 0, None, 0),
 (20, 'ARE', 'FLOAT', 0, None, 0),
 (21, 'ALXN', 'FLOAT', 0, None, 0),
 (22, 'ALGN', 'FLOAT', 0, None, 0),
 (23, 'ALLE', 'FLOAT', 0, None, 0),
 (24, 'AGN', 'FLOAT', 0, None, 0),
 (25, 'ADS', 'FLOAT', 0, None, 0),
 (26, 'LNT', 'FLOAT', 0, None, 0),
 (27, 'ALL', 'FLOAT', 0, None, 0),
 (28, 'GOOGL', 'FLOAT'

In [228]:
dff = pd.read_sql_table('price_close', con=engine)

In [229]:
dff

,index,date,MMM,AOS,ABT,ABBV,ACN,ATVI,AYI,ADBE,...,WU,WY,WHR,WMB,WYNN,XEL,XRX,XLNX,XYL,YUM
0,0,2019-07-12,173.34,45.19,83.92,70.28,196.21,47.200,130.64,309.42,...,20.50,25.54,146.42,28.85,133.89,60.80,35.37,120.32,82.50,111.05
1,1,2019-07-11,169.04,44.32,85.77,71.24,194.67,47.340,128.01,307.60,...,20.41,25.81,142.63,28.71,130.65,61.19,34.92,118.37,81.15,111.50
2,2,2019-07-10,165.70,44.77,85.38,71.65,191.71,46.840,127.95,306.98,...,20.55,26.31,142.08,28.82,130.34,61.17,35.48,117.49,80.39,110.98
3,3,2019-07-09,165.70,45.06,85.27,71.24,191.09,46.370,129.98,304.20,...,20.65,26.29,142.80,28.74,133.73,60.89,35.95,117.34,80.98,110.49
4,4,2019-07-08,169.19,45.21,85.09,71.22,190.31,46.470,131.47,302.92,...,20.35,26.43,144.53,28.44,133.06,60.85,35.66,115.93,81.70,110.05
5,5,2019-07-05,172.00,47.01,85.42,72.99,191.39,48.150,130.69,303.98,...,20.35,26.61,144.66,28.35,132.81,60.69,36.44,116.11,82.84,110.24
6,6,2019-07-03,174.98,46.79,85.64,74.96,190.97,48.660,135.17,305.70,...,20.25,26.85,146.23,28.39,133.38,60.71,36.48,116.39,83.33,110.98
7,7,2019-07-02,174.02,46.99,84.95,74.23,188.60,47.980,129.95,301.39,...,20.16,26.52,144.84,28.20,132.97,60.15,35.64,117.60,83.17,110.90
8,8,2019-07-01,174.51,46.93,84.49,73.40,186.87,47.500,141.02,300.97,...,19.98,26.40,145.58,28.06,131.25,59.43,36.24,121.15,84.41,109.49
9,9,2019-06-28,173.34,47.16,84.10,72.72,184.77,47.200,137.91,294.65,...,19.89,26.34,142.36,28.04,123.99,59.49,35.41,117.92,83.64,110.67
